In [1]:
import cv2
import mediapipe as mp
import os
import numpy as np
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib

In [2]:
# Inicializando o MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Pasta com os vídeos
pasta_videos = './pasta2'

# Listas para armazenar os dados e os rótulos
dados = []
rotulos = []

# Dicionário para armazenar as coordenadas por palavra
coordenadas_por_palavra = {'direita': {}, 'esquerda': {}}

In [3]:
for subpasta in os.listdir(pasta_videos):
    for video in os.listdir(os.path.join(pasta_videos, subpasta)):
        # Lendo o vídeo
        cap = cv2.VideoCapture(os.path.join(pasta_videos, subpasta, video))
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Convertendo a cor da imagem
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # Processando a imagem
            result = hands.process(image)
            
            if result.multi_hand_landmarks and result.multi_handedness:
                for hand_landmarks, handedness in zip(result.multi_hand_landmarks, result.multi_handedness):
                    # Identificar se a mão é direita ou esquerda
                    hand_label = handedness.classification[0].label.lower()  # 'left' ou 'right'
                    if hand_label == 'right':
                        hand_label = 'direita'
                    else:
                        hand_label = 'esquerda'
                    
                    # Extraindo as coordenadas
                    coords = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in hand_landmarks.landmark]).flatten())
                    
                    # Adicionando as coordenadas à lista de dados
                    dados.append(coords)
                    # Adicionando o rótulo à lista de rótulos
                    rotulos.append(subpasta)
                    
                    # Armazenando as coordenadas por palavra e mão
                    if subpasta not in coordenadas_por_palavra[hand_label]:
                        coordenadas_por_palavra[hand_label][subpasta] = []
                    coordenadas_por_palavra[hand_label][subpasta].append(tuple(coords))

# Selecionando a coordenada mais frequente por palavra e por mão
coordenadas_selecionadas = {'direita': {}, 'esquerda': {}}
for hand_label in ['direita', 'esquerda']:
    for palavra, coords_lista in coordenadas_por_palavra[hand_label].items():
        # Contar a frequência de cada conjunto de coordenadas
        contador = Counter(coords_lista)
        # Selecionar a coordenada mais frequente
        mais_frequente = contador.most_common(1)[0][0]
        coordenadas_selecionadas[hand_label][palavra] = mais_frequente

# Preparando os dados para treinamento
dados_filtrados = []
rotulos_filtrados = []
for hand_label in ['direita', 'esquerda']:
    for palavra, coords in coordenadas_selecionadas[hand_label].items():
        dados_filtrados.append(coords)
        rotulos_filtrados.append(palavra)

# Convertendo as listas em arrays numpy
dados = np.array(dados_filtrados)
rotulos = np.array(rotulos_filtrados)

# Balanceamento dos dados
min_amostras = min(Counter(rotulos).values())
dados_balanceados = []
rotulos_balanceados = []
for palavra in coordenadas_selecionadas['direita'].keys():
    indices = [i for i, r in enumerate(rotulos) if r == palavra]
    indices_selecionados = np.random.choice(indices, min_amostras, replace=False)
    dados_balanceados.extend(dados[indices_selecionados])
    rotulos_balanceados.extend(rotulos[indices_selecionados])

dados_balanceados = np.array(dados_balanceados)
rotulos_balanceados = np.array(rotulos_balanceados)

# Dividindo os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(dados_balanceados, rotulos_balanceados, test_size=0.2, random_state=42)

# Treinando o modelo com as coordenadas selecionadas
modelo = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
modelo.fit(X_train, y_train)

# Avaliando o modelo
score = modelo.score(X_test, y_test)
print(f'Acurácia do modelo: {score*100:.2f}%')

# Salvando o modelo
joblib.dump(modelo, 'modelo.pkl')

c:\Users\danil\Documents\GitHub\TCC2_modelo_IA\myenv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Acurácia do modelo: 0.00%


['modelo.pkl']